In [1]:
#Package import
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.model_selection import train_test_split

In [29]:
#Data values load
dementia_data = pd.read_csv(r"C:\\Joshna\dementia_dataset.csv")
dementia_data.head(7)

,Subject_ID,MRI_ID,Group,Visit,MR_Delay,M_F,Hand,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,OAS2_0001,OAS2_0001_MR1,Nondemented,1,0,M,R,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,OAS2_0001,OAS2_0001_MR2,Nondemented,2,457,M,R,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,OAS2_0002,OAS2_0002_MR1,Demented,1,0,M,R,75,12,NaN,23.0,0.5,1678,0.736,1.046
3,OAS2_0002,OAS2_0002_MR2,Demented,2,560,M,R,76,12,NaN,28.0,0.5,1738,0.713,1.010
4,OAS2_0002,OAS2_0002_MR3,Demented,3,1895,M,R,80,12,NaN,22.0,0.5,1698,0.701,1.034
5,OAS2_0004,OAS2_0004_MR1,Nondemented,1,0,F,R,88,18,3.0,28.0,0.0,1215,0.710,1.444
6,OAS2_0004,OAS2_0004_MR2,Nondemented,2,538,F,R,90,18,3.0,27.0,0.0,1200,0.718,1.462


In [51]:
#Information about the dataset
dementia_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 373 entries, 0 to 372
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Subject_ID  373 non-null    object 
 1   MRI_ID      373 non-null    object 
 2   Group       373 non-null    object 
 3   Visit       373 non-null    int64  
 4   MR_Delay    373 non-null    int64  
 5   M_F         373 non-null    object 
 6   Hand        373 non-null    object 
 7   Age         373 non-null    int64  
 8   EDUC        373 non-null    int64  
 9   SES         373 non-null    float64
 10  MMSE        373 non-null    float64
 11  CDR         373 non-null    float64
 12  eTIV        373 non-null    int64  
 13  nWBV        373 non-null    float64
 14  ASF         373 non-null    float64
 15  type        373 non-null    int64  
dtypes: float64(5), int64(6), object(5)
memory usage: 46.8+ KB


In [30]:
#Fill null values with 0
dementia_data=dementia_data.fillna(0)

In [31]:
#Changing the dependent categorical variable to a numeric variable 
def dep_trans(form):
    if form=='Nondemented':
        return 0
    elif form=='Demented':
        return 1
    elif form=='Converted':
        return 2

In [32]:
#Creating a new dependent variable of numeric type
dementia_data['type'] = dementia_data['Group'].apply(dep_trans)

In [33]:
#Creating feature var and target var
feat = dementia_data.drop(['Subject_ID','MRI_ID','M_F','Hand','Group','type'],axis=1)
targ = dementia_data.loc[:,'type']

In [34]:
feat

,Visit,MR_Delay,Age,EDUC,SES,MMSE,CDR,eTIV,nWBV,ASF
0,1,0,87,14,2.0,27.0,0.0,1987,0.696,0.883
1,2,457,88,14,2.0,30.0,0.0,2004,0.681,0.876
2,1,0,75,12,0.0,23.0,0.5,1678,0.736,1.046
3,2,560,76,12,0.0,28.0,0.5,1738,0.713,1.010
4,3,1895,80,12,0.0,22.0,0.5,1698,0.701,1.034
...,...,...,...,...,...,...,...,...,...,...
368,2,842,82,16,1.0,28.0,0.5,1693,0.694,1.037
369,3,2297,86,16,1.0,26.0,0.5,1688,0.675,1.040
370,1,0,61,13,2.0,30.0,0.0,1319,0.801,1.331
371,2,763,63,13,2.0,30.0,0.0,1327,0.796,1.323


In [35]:
#Dataset division
X_train, X_test, Y_train, Y_test = train_test_split(feat,targ,test_size=0.20)

In [36]:
#Tensorflow form of variables
indepndnt = [tf.feature_column.numeric_column('Visit'),
            tf.feature_column.numeric_column('MR_Delay'),
            tf.feature_column.numeric_column('Age'),
            tf.feature_column.numeric_column('EDUC'),
            tf.feature_column.numeric_column('SES'),
            tf.feature_column.numeric_column('MMSE'),
            tf.feature_column.numeric_column('CDR'),
            tf.feature_column.numeric_column('eTIV'),
            tf.feature_column.numeric_column('nWBV'),
            tf.feature_column.numeric_column('ASF')           
           ]

In [37]:
#Model parameter define step
def parms_model(epochs,batches,shuffle):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x=X_train,
        y=Y_train,
        batch_size=batches,
        num_epochs=epochs,
       shuffle=shuffle
    )

In [38]:
#Model validation step
def valid_model(epochs,batches,shuffle):
    return tf.compat.v1.estimator.inputs.pandas_input_fn(
        x=X_test,
        y=Y_test,
        batch_size=batches,
        num_epochs=epochs,
        shuffle=shuffle
    )

In [39]:
#Forming the ANN model with defined parameters
ANN_dem_model = tf.estimator.DNNClassifier(n_classes=3,
                                       hidden_units=[1024,512,256,32,3],
                                       optimizer='Adam',
                                       feature_columns=indepndnt,
                                       dropout=0.27,
                                       activation_fn=tf.nn.relu
                                      )

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\joshna\\AppData\\Local\\Temp\\tmpxzh3zgjp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [47]:
#Training the ANN model with the model parameters
ANN_dem_model.train(input_fn=parms_model(100,128,True),steps=900)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\joshna\AppData\Local\Temp\tmpxzh3zgjp\model.ckpt-466
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 466...
INFO:tensorflow:Saving checkpoints for 466 into C:\Users\joshna\AppData\Local\Temp\tmpxzh3zgjp\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 466...
INFO:tensorflow:loss = 1.0089071, step = 466
INFO:tensorflow:global_step/sec: 71.037
INFO:tensorflow:loss = 0.9348132, step = 566 (1.413 sec)
INFO:tensorflow:global_step/sec: 78.2553
INFO:tensorflow:loss = 0.9602465, step = 666 (1.274 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 699...
INFO:tensorflow:Saving checkpoints for 699 into C:\Users\joshna\AppData\Local\Temp

In [50]:
#Performance check of the ANN model
ANN_dem_model.evaluate(input_fn=valid_model(100,128,True),steps=900)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-03-03T18:35:04
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\joshna\AppData\Local\Temp\tmpxzh3zgjp\model.ckpt-699
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.57411s
INFO:tensorflow:Finished evaluation at 2023-03-03-18:35:05
INFO:tensorflow:Saving dict for global step 699: accuracy = 0.53333336, average_loss = 0.96190774, global_step = 699, loss = 0.9619539
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 699: C:\Users\joshna\AppData\Local\Temp\tmpxzh3zgjp\model.ckpt-699


{'accuracy': 0.53333336,
 'average_loss': 0.96190774,
 'loss': 0.9619539,
 'global_step': 699}

In [44]:
#Forming the ANN model with the features of the Linear Classifier model
ANN_linear_dem_model = tf.estimator.DNNLinearCombinedClassifier(n_classes=3,dnn_dropout=0.27,dnn_optimizer='Adam',dnn_feature_columns=indepndnt,dnn_hidden_units=[1024,512,256,32,3],linear_feature_columns=indepndnt,dnn_activation_fn='relu')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\joshna\\AppData\\Local\\Temp\\tmpya4gc0qi', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [45]:
#Training the ANN model with the features of the Linear Classifier model
ANN_linear_dem_model.train(input_fn=parms_model(100,128,True),steps=900)

INFO:tensorflow:Calling model_fn.


C:\Users\joshna\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\joshna\AppData\Local\Temp\tmpya4gc0qi\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 70.44851, step = 0
INFO:tensorflow:global_step/sec: 69.3832
INFO:tensorflow:loss = 0.89021534, step = 100 (1.441 sec)
INFO:tensorflow:global_step/sec: 79.2563
INFO:tensorflow:loss = 0.82194686, step = 200 (1.264 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 233...
INFO:tensorflow:Saving checkpoints for 233 into C:\Users\joshna\AppData\Local\Temp\tmpya4gc0qi\model.ckpt.
I

In [46]:
#Performance check of the ANN model with the features of the Linear Classifier model
ANN_linear_dem_model.evaluate(input_fn=valid_model(100,128,True),steps=900)

INFO:tensorflow:Calling model_fn.


C:\Users\joshna\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1700: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2023-03-03T18:04:40
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\joshna\AppData\Local\Temp\tmpya4gc0qi\model.ckpt-233
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.71251s
INFO:tensorflow:Finished evaluation at 2023-03-03-18:04:40
INFO:tensorflow:Saving dict for global step 233: accuracy = 0.68, average_loss = 0.8763111, global_step = 233, loss = 0.8764983
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 233: C:\Users\joshna\AppData\Local\Temp\tmpya4gc0qi\model.ckpt-233


{'accuracy': 0.68,
 'average_loss': 0.8763111,
 'loss': 0.8764983,
 'global_step': 233}